<a href="https://colab.research.google.com/github/stasrodov/self-rag-eval/blob/main/gemma2_is_relevant_passage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## This is part of Self-RAG evaluation final project
### See main section in 'self_rag_eval_project.ipynb' in github



### Loading the Attribute-QA dataset and Gemma-2 model

In [ ]:
!pip install -U transformers
!pip install accelerate
!pip install pandas

In [ ]:
# Constants:
ATTR_QA_PATH = 'Attributed-QA'
RATINGS_FILENAME = 'ratings'
SELF_RAG_PATH = 'self-rag'

In [ ]:
from zipfile import ZipFile
import pandas as pd

!git clone https://github.com/google-research-datasets/Attributed-QA.git

with ZipFile(f'{ATTR_QA_PATH}/{RATINGS_FILENAME}.zip') as ratings_zip:
  ratings_zip.extract(f'{RATINGS_FILENAME}.csv', ATTR_QA_PATH)

ratings_df = pd.read_csv(f'{ATTR_QA_PATH}/{RATINGS_FILENAME}.csv')

# Filter rows without human rating:
ratings_df = ratings_df[ratings_df['human_rating'] != '-']

# Filter unneeded columns:
columns_all = ratings_df.columns.values
columns_needed = ['question', 'passage', 'human_rating', 'attribution'] # 'answer' !!
columns_unneeded = set(columns_all) - set(columns_needed)
ratings_df = ratings_df.drop(columns=columns_unneeded)

# Filter duplicate rows:
ratings_df = ratings_df.drop_duplicates()
# ratings_df # 7121 rows

# Filter rows with different 'human_rating' values while the rest is the same:
columns_no_hr = [c for c in columns_needed if c != 'human_rating']
ratings_df = ratings_df.drop_duplicates(subset=columns_no_hr, keep=False)

ratings_df.to_csv(f'{ATTR_QA_PATH}/{RATINGS_FILENAME}_filtered.csv', index=False)
ratings_df # 5785 rows

fatal: destination path 'Attributed-QA' already exists and is not an empty directory.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


,question,passage,human_rating,attribution
0,who played hyde in league of extraordinary gen...,Title: Jason Flemyng\nSection: Television and ...,Y,http://en.wikipedia.org/wiki/Jason_Flemyng#Jas...
7,who signed the largest on the declaration of i...,Title: United States Declaration of Independen...,Y,http://en.wikipedia.org/wiki/United_States_Dec...
16,when was the last time the carolina hurricanes...,Title: 2009 Stanley Cup playoffs\nSection: Con...,Y,http://en.wikipedia.org/wiki/2009_Stanley_Cup_...
21,where was 2017 beauty and the beast filmed,Title: Beauty and the Beast (2017 film)\n\nA l...,Y,http://en.wikipedia.org/wiki/Beauty_and_the_Be...
23,when does the next warrior book come out,Title: Adonal Foyle\n\nAdonal David Foyle (bor...,N,http://en.wikipedia.org/wiki/Adonal_Foyle#Adon...
...,...,...,...,...
81842,the road that connects the tombs is called,Title: Valley of the Kings (Tibet)\n\nThe Vall...,N,http://en.wikipedia.org/wiki/Valley_of_the_Kin...
81843,when was the miraculous journey of edward tula...,Title: The Miraculous Journey of Edward Tulane...,N,http://en.wikipedia.org/wiki/The_Miraculous_Jo...
81844,where was the remake of dirty dancing filmed,"Title: Asheville, North Carolina\nSection: In ...",N,"http://en.wikipedia.org/wiki/Asheville,_North_..."
81845,how long is a whale shark in meters,Title: List of largest fish\nSection: Cartilag...,N,http://en.wikipedia.org/wiki/List_of_largest_f...


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import os

access_token = os.getenv('RUNPOD_SECRET_huggingface')
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-27b", token=access_token)
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-27b",
    device_map="auto",
    torch_dtype=torch.bfloat16,
    token=access_token)

tokenizer_config.json:   0%|          | 0.00/40.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/892 [00:00<?, ?B/s]

...

model-00022-of-00024.safetensors:   0%|          | 0.00/4.53G [00:00<?, ?B/s]

model-00023-of-00024.safetensors:   0%|          | 0.00/4.53G [00:00<?, ?B/s]

model-00024-of-00024.safetensors:   0%|          | 0.00/4.30G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/24 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [ ]:
def gemma_get_response(model, tokenizer, prompt, max_new_tokens=1024):
    input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**input_ids, max_new_tokens=max_new_tokens)
    return tokenizer.decode(outputs[0])

In [ ]:
prompt = 'write poem about machine learning'
response_txt = gemma_get_response(model, tokenizer, prompt, max_new_tokens=200)
print(response_txt)

<bos>write poem about machine learning

Machine learning is a field of artificial intelligence that uses statistical techniques to give computer systems the ability to "learn" (i.e., progressively improve performance on a specific task) from data, without being explicitly programmed.

Machine learning algorithms build a mathematical model based on sample data, known as "training data", in order to make predictions or decisions without being explicitly programmed to perform the task. These algorithms are used in a wide variety of applications, such as email filtering, computer vision, speech recognition, natural language processing, and medical diagnosis.

Machine learning is closely related to and often overlaps with computational statistics, which also focuses on prediction-making using computers. The study of mathematical optimization delivers methods, theory and application domains to the field of machine learning. Data mining is a related field of study, focusing on exploratory dat

### Trying Gemma-2 to imitate the behavior of self-rag's 'Relevant' Tags

#### Zero-shot, with/without CoT:
---
CoT didn't really work.  
Zero-shot (withot CoT) led to poor results.

In [ ]:
TAG_YES = '<Answer>Yes</Answer>'
TAG_NO = '<Answer>No</Answer>'

def create_prompt_is_relevant_passage(question, passage):
  return f"""You are an expert in trivia questions. Decide if the following passage <Passage> is relevant for answering the following question <Question>.
Explain your reasoning process, then write the final answer "Yes" or "No" between the tags <Answer> and </Answer>.
Here is the question <Question>: <Question>{question}</Question>
Here is the passage <Passage>: <Passage>{passage}</Passage>\n"""

def create_prompt_is_relevant_passage_no_cot(question, passage):
  return f"""You are an expert in trivia questions. Decide if the following passage <Passage> is relevant for answering the following question <Question>.
Write only "Yes" or "No" between the tags <Answer> and </Answer> AND NOTHING MORE.
Here is the question <Question>: <Question>{question}</Question>
Here is the passage <Passage>: <Passage>{passage}</Passage>\n"""

In [ ]:
from tqdm.notebook import tqdm

ratings_df = pd.read_csv(f'{ATTR_QA_PATH}/{RATINGS_FILENAME}_filtered.csv')

total = len(ratings_df)
gemma2_preds = []
gemma2_ratings = []

for i, row in  tqdm(ratings_df.iterrows(), desc="Querying Claude...", total=total):
  prompt = create_prompt_is_relevant_passage_no_cot(row.question, row.passage)
  response_txt = gemma_get_response(model, tokenizer, prompt, max_new_tokens=200)
  gemma2_preds.append(response_txt)
  if TAG_YES in response_txt:
    gemma2_ratings.append('Y')
  elif TAG_NO in response_txt:
    gemma2_ratings.append('N')
  else:
    gemma2_ratings.append('-')

In [ ]:
ratings_2769_df = ratings_df.head(2769)
ratings_2769_df['gemma2_rating'] = gemma2_ratings
ratings_2769_df['gemma2_pred'] = gemma2_preds
ratings_2769_df.to_csv(f'{ATTR_QA_PATH}/{RATINGS_FILENAME}_relevant_gemma_2769.csv', index=False)

In [ ]:
# Statistics:
ratings_gemma_df = pd.read_csv(f'{ATTR_QA_PATH}/{RATINGS_FILENAME}_relevant_gemma_2769.csv')

total = len(ratings_gemma_df)
yes = len(ratings_gemma_df[ratings_gemma_df['gemma2_rating'] == 'Y'])
no =  len(ratings_gemma_df[ratings_gemma_df['gemma2_rating'] == 'N'])
none = len(ratings_gemma_df[ratings_gemma_df['gemma2_rating'] == '-'])
print(f'Total: {total}, #Yes: {yes}, #No: {no}, #-: {none}')

same = len(ratings_gemma_df[ratings_gemma_df['gemma2_rating'] == ratings_gemma_df['human_rating']])
print(f'Same: {same}, Different: {total - same}, %Same: {round(same / total * 100, 2)}')

Total: 2769, #Yes: 2155, #No: 68, #-: 546
Same: 950, Different: 1819, %Same: 34.31


#### Few-shot:
---
Reasonable results, better than zero-shot with/without CoT.

In [ ]:
# Creating few-shot prompts:
ratings_df_tail = pd.read_csv(f'{ATTR_QA_PATH}/{RATINGS_FILENAME}_filtered.csv').tail(2000)
ratings_yes_df = ratings_df_tail[ratings_df_tail['human_rating'] == 'Y']
ratings_no_df = ratings_df_tail[ratings_df_tail['human_rating'] == 'N']
ratings_yes_no_df =  pd.concat([ratings_yes_df.head(2), ratings_no_df.head(2)])

dem_len = len(ratings_yes_no_df)
TAG_YES = f'<A{dem_len + 1}>Yes</A{dem_len + 1}>'
TAG_NO = f'<A{dem_len + 1}>No</A{dem_len + 1}>'

few_shot_prompt_prefix = ''
for i in range(dem_len):
    n = i + 1
    row = ratings_yes_no_df.iloc[i]
    ans = 'Yes' if row.human_rating == 'Y' else 'No'
    few_shot_prompt_prefix += f'<Q{n}>{row.question}</Q{n}><P{n}>{row.passage}</P{n}><A{n}>{ans}</A{n}>\n'

def create_prompt_is_relevant_passage_few_shot(question, passage, demos=few_shot_prompt_prefix, dem_len=dem_len):
    n = dem_len + 1
    return demos + f'<Q{n}>{question}</Q{n}><P{n}>{passage}</P{n}><A{n}>'

In [ ]:
# Some exploration with the prompt:
ratings_gemma_df = pd.read_csv(f'{ATTR_QA_PATH}/{RATINGS_FILENAME}_relevant_gemma_2769.csv')
row = ratings_gemma_df.iloc[0]
prompt = create_prompt_is_relevant_passage_few_shot(row.question, row.passage)
print(prompt)

<Q1>which approach to psychology focuses on the body especially the brain and nervous system</Q1><P1>Title: Psychology of learning
Section: Psychology of learning theories, Neuroscience

The neuroscience of learning focuses on the relationships among the central nervous system, learning, and behavior. This central nervous system (CNS) is composed of the brain and spinal cord which are responsible for controlling behavior. This differs from the autonomic nervous system which relates with more autonomous functions such as respiration and digestion. Within the CNS, there are neurons, which send messages within and from the brain to the rest of body through electrical and chemical signals. The CNS also has glial cells which assist neurons in the communication process. These neurons communicate and form connections, also called consolidation, to form a person's cognition. Hence, learning, from a neuroscience perspective, focuses on how the organization of this neural network changes through

In [ ]:
response_txt = gemma_get_response(model, tokenizer, prompt, max_new_tokens=200)
print(response_txt)
print(TAG_YES in response_txt)
print(TAG_NO in response_txt)

<bos><Q1>which approach to psychology focuses on the body especially the brain and nervous system</Q1><P1>Title: Psychology of learning
Section: Psychology of learning theories, Neuroscience

The neuroscience of learning focuses on the relationships among the central nervous system, learning, and behavior. This central nervous system (CNS) is composed of the brain and spinal cord which are responsible for controlling behavior. This differs from the autonomic nervous system which relates with more autonomous functions such as respiration and digestion. Within the CNS, there are neurons, which send messages within and from the brain to the rest of body through electrical and chemical signals. The CNS also has glial cells which assist neurons in the communication process. These neurons communicate and form connections, also called consolidation, to form a person's cognition. Hence, learning, from a neuroscience perspective, focuses on how the organization of this neural network changes th

In [ ]:
# Let's try to find the generated question 'what is the name of the first episode of the office' in our dataset:
ratings_df[ratings_df['question'] == 'what is the name of the first episode of the office']

,question,passage,human_rating,attribution


In [ ]:
ratings_df[ratings_df['question'].str.contains('office')]

# Checked also is the unfiltered CSV (not in this code).
# Seems that the question does not appear in our dataset, but in might appear in googl's NQ dataset, which the CSV is derived from.

,question,passage,human_rating,attribution
15691,how long prime minister stay in office canada,Title: List of prime ministers of Canada by ti...,N,http://en.wikipedia.org/wiki/List_of_prime_min...
17841,how long prime minister stay in office canada,Title: At Her Majesty's pleasure\nSection: Ser...,Y,http://en.wikipedia.org/wiki/At_Her_Majesty's_...
25614,how long prime minister stay in office canada,Title: Prime Minister of Canada\nSection: Term...,N,http://en.wikipedia.org/wiki/Prime_Minister_of...
33854,how long prime minister stay in office canada,"Title: Term of office\nSection: Canada, House ...",N,http://en.wikipedia.org/wiki/Term_of_office#Te...
35284,how long prime minister stay in office canada,Title: Prime Minister of Canada\nSection: Term...,Y,http://en.wikipedia.org/wiki/Prime_Minister_of...
55720,how long prime minister stay in office canada,Title: List of prime ministers of Canada by ti...,N,http://en.wikipedia.org/wiki/List_of_prime_min...


In [ ]:
from tqdm.notebook import tqdm

ratings_gemma_df = pd.read_csv(f'{ATTR_QA_PATH}/{RATINGS_FILENAME}_relevant_gemma_2769.csv')

# Finish what's left:
ratings_gemma_df = ratings_df.tail(2119)
# -------------------------------------------------------------------

total = len(ratings_gemma_df)
gemma2_preds = []
gemma2_ratings = []

for i, row in  tqdm(ratings_gemma_df.iterrows(), desc="Querying Claude...", total=total):
  prompt = create_prompt_is_relevant_passage_few_shot(row.question, row.passage)
  response_txt = gemma_get_response(model, tokenizer, prompt, max_new_tokens=5)
  gemma2_preds.append(response_txt)
  if TAG_YES in response_txt:
    gemma2_ratings.append('Y')
  elif TAG_NO in response_txt:
    gemma2_ratings.append('N')
  else:
    gemma2_ratings.append('-')


In [ ]:
ratings_gemma_df['gemma2_rating'] = gemma2_ratings
ratings_gemma_df['gemma2_pred'] = gemma2_preds
ratings_gemma_df.to_csv(f'{ATTR_QA_PATH}/{RATINGS_FILENAME}_relevant_gemma_2119_few-shot.csv', index=False)

In [ ]:
# Merge with results from previous run:
ratings_head_df = pd.read_csv(f'{ATTR_QA_PATH}/{RATINGS_FILENAME}_relevant_gemma_650_few-shot.csv')
ratings_tail_df = pd.read_csv(f'{ATTR_QA_PATH}/{RATINGS_FILENAME}_relevant_gemma_2119_few-shot.csv')
ratings_gemma_merged_df = pd.concat([ratings_head_df, ratings_tail_df])
ratings_gemma_merged_df.to_csv(f'{ATTR_QA_PATH}/{RATINGS_FILENAME}_relevant_gemma_merged_few-shot.csv', index=False)

In [ ]:
# Final merged statistics:
ratings_gemma_fs_df = pd.read_csv(f'{ATTR_QA_PATH}/{RATINGS_FILENAME}_relevant_gemma_merged_few-shot.csv')

total = len(ratings_gemma_fs_df)
yes = len(ratings_gemma_fs_df[ratings_gemma_fs_df['gemma2_rating'] == 'Y'])
no =  len(ratings_gemma_fs_df[ratings_gemma_fs_df['gemma2_rating'] == 'N'])
none = len(ratings_gemma_fs_df[ratings_gemma_fs_df['gemma2_rating'] == '-'])
print(f'Total: {total}, #Yes: {yes}, #No: {no}, #-: {none}')

same = len(ratings_gemma_fs_df[ratings_gemma_fs_df['gemma2_rating'] == ratings_gemma_fs_df['human_rating']])
print(f'Same: {same}, Different: {total - same}, %Same: {round(same / total * 100, 2)}')

Total: 2769, #Yes: 746, #No: 2018, #-: 5
Same: 1809, Different: 960, %Same: 65.33
